In [1]:
from torch_geometric.datasets import Planetoid
import torch
from torch_geometric.data import Data

In [2]:
dataset_cora = Planetoid(root='./cora/', name='Cora')
# dataset = Planetoid(root='./citeseer',name='Citeseer')
# dataset = Planetoid(root='./pubmed/',name='Pubmed')
print(dataset_cora[0])

Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
from torch_geometric.nn import SAGEConv
from torch_geometric.nn import GATConv

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = SAGEConv(dataset_cora.num_node_features, 16, 'lstm')
        self.conv2 = SAGEConv(16, dataset_cora.num_classes, 'lstm')

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.softmax(x, dim=1)

        return x
    
class GAT(nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
        self.conv1 = GATConv(dataset_cora.num_node_features, 16, heads=2)
        self.conv2 = GATConv(2*16, dataset_cora.num_classes, heads=1)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.softmax(x, dim=1)

        return x

In [7]:
model = GAT()
print(model)

GAT(
  (conv1): GATConv(1433, 16, heads=2)
  (conv2): GATConv(32, 7, heads=1)
)


In [8]:
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
print(device)
model.to(device)
data = dataset_cora[0].to(device)
print(data)

cuda:1
Data(x=[2708, 1433], edge_index=[2, 10556], y=[2708], train_mask=[2708], val_mask=[2708], test_mask=[2708])


In [11]:
import torch.optim as optim
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min')

In [12]:
model.train()
for epoch in range(200):
    out = model(data)
    loss = criterion(out[data.train_mask], data.y[data.train_mask])

    optimizer.zero_grad()
    loss.backward()
    scheduler.step(loss)
    optimizer.step()

    _, pred = torch.max(out[data.train_mask], dim=1)
    correct = (pred == data.y[data.train_mask]).sum().item()
    acc = correct/data.train_mask.sum().item()

    print('Epoch {:03d} train_loss: {:.4f} train_acc: {:.4f}'.format(
        epoch, loss.item(), acc))

Epoch 000 train_loss: 1.1844 train_acc: 0.9857
Epoch 001 train_loss: 1.1903 train_acc: 0.9857
Epoch 002 train_loss: 1.1836 train_acc: 0.9929
Epoch 003 train_loss: 1.1756 train_acc: 0.9929
Epoch 004 train_loss: 1.1865 train_acc: 0.9857
Epoch 005 train_loss: 1.1806 train_acc: 0.9929
Epoch 006 train_loss: 1.1847 train_acc: 0.9857
Epoch 007 train_loss: 1.1887 train_acc: 0.9857
Epoch 008 train_loss: 1.1868 train_acc: 0.9857
Epoch 009 train_loss: 1.1761 train_acc: 0.9929
Epoch 010 train_loss: 1.1771 train_acc: 1.0000
Epoch 011 train_loss: 1.1847 train_acc: 0.9857
Epoch 012 train_loss: 1.1698 train_acc: 1.0000
Epoch 013 train_loss: 1.1723 train_acc: 1.0000
Epoch 014 train_loss: 1.1733 train_acc: 1.0000
Epoch 015 train_loss: 1.1743 train_acc: 1.0000
Epoch 016 train_loss: 1.1744 train_acc: 1.0000
Epoch 017 train_loss: 1.1793 train_acc: 0.9929
Epoch 018 train_loss: 1.1695 train_acc: 1.0000
Epoch 019 train_loss: 1.1751 train_acc: 1.0000
Epoch 020 train_loss: 1.1778 train_acc: 1.0000
Epoch 021 tra